In [1]:
# !wget https://github.com/huseinzol05/malay-dataset/blob/master/news/news-30k/news-30k.json.zip?raw=true
# !unzip news-30k.json.zip?raw=true
# !rm news-30k.json.zip?raw=true

In [2]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/news/news-2020-09-11.zip
# !unzip news-2020-09-11.zip
# !rm news-2020-09-11.zip

In [3]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [4]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('mesolitica/finetune-summarization-t5-base-standard-bahasa-cased')
model = T5ForConditionalGeneration.from_pretrained('mesolitica/finetune-summarization-t5-base-standard-bahasa-cased')

In [53]:
small_model = T5ForConditionalGeneration.from_pretrained('mesolitica/finetune-summarization-t5-small-standard-bahasa-cased')

In [54]:
_ = model.cuda()
_ = small_model.cuda()

In [6]:
import json

with open('news-30k.json') as fopen:
    news = json.load(fopen)

In [74]:
from glob import glob

files = glob('news/*.json')
len(files)

976

In [7]:
len(news)

29855

In [8]:
news[0]

{'title': 'Ibu saudara Haziq terharu sokongan rakyat Malaysia',
 'url': 'https://www.themalaysianinsight.com/bahasa/s/142491',
 'news': 'themalaysianinsight',
 'language': 'malay',
 'top-image': 'https://www.themalaysianinsight.com/resources/stories_images/142491/perhimpunanan_solidarity_kedamaian_03__full.jpg',
 'text': 'WARIS keluarga Allahyarham Muhammad Haziq Mohd Tarmizi, 17, yang terkorban dalam tragedi tembakan di Christchurch, New Zealand, pada 15 Mac lepas, melahirkan rasa terharu akan sokongan diberi rakyat Malaysia semasa perhimpunan Solidariti Kedamaian.\n\nZarina Shuib , ibu saudara Muhammad Haziq, memanjatkan kesyukuran kepada Allah SWT kerana berkesempatan menyertai rakyat Malaysia dalam perhimpunan itu di Kuala Lumpur hari ini.',
 'date': '2019-03-23T03:52:02',
 'date_utc': '2019-03-22T19:52:02'}

In [9]:
from unidecode import unidecode
import re

# minimum cleaning, just simply to remove newlines.
def cleaning(string):
    string = string.replace('\n', ' ')
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

In [60]:
selected = []
already = set()

for n in news:
    if n['url'] not in already:
        selected.append(n)
        already.add(n['url'])
        
len(selected), len(news)

(24793, 29855)

In [72]:
s = cleaning(selected[2]['text'])
s

'PULAU PINANG 23 Mac - Seorang pekerja di sebuah kilang di Bayan Lepas di sini kerugian RM24,000 setelah terpedaya dengan sindiket scam yang menyamar sebagai anggota polis kelmarin. Ketua Polis Daerah Barat Daya, Superintendan A. A. Anbalagan, berkata, mangsa seorang wanita berusia 43 tahun itu telah menerima panggilan kononnya dari kakitangan Hospital Besar Pulau Pinang yang memperkenalkan diri dengan nama Christopher Anak Anor kira-kira pukul 2 petang kelmarin. “Christopher menyatakan bahawa seorang individu nama Lee Song telah menggunakan kad pengenalan mangsa untuk mengambil ubat terkawal di Hospital Seberang Prai dan Hospital Pulau Pinang namun mangsa menafikan mengenali individu tersebut. “Christopher meminta mangsa membuat laporan polis dan mangsa telah disambungkan menerusi telefon untuk bercakap dengan Sarjan Tony yang kemudiannya menyatakan bahawa nama mangsa telah disenarai hitam kerana terlibat dengan jenayah senjata api,” katanya di sini hari ini. Menurut Anbalagan mangsa 

In [69]:
from malaya.text.rouge import postprocess_summary

def postprocess(s, t):
    t = t.replace('£', 'RM')
    return postprocess_summary(s, t)

minimum_len = 15

In [76]:
with open(files[0]) as fopen:
    data = json.load(fopen)

In [77]:
len(data)

256

In [79]:
s = cleaning(data[0]['text'])

In [82]:
%%time

input_ids = tokenizer.encode(f'ringkasan: {s}', return_tensors = 'pt').cuda()
outputs = model.generate(input_ids,
    max_length=256)

small_outputs = small_model.generate(input_ids,
    max_length=256)

t = tokenizer.batch_decode(outputs, skip_special_tokens=True)
t.extend(tokenizer.batch_decode(small_outputs, skip_special_tokens=True))
t = [t_ for t_ in t if len(t_.split()) >= minimum_len]
t = [postprocess(s, t_) for t_ in t]
t = list(set(t))
t

CPU times: user 780 ms, sys: 10.1 ms, total: 790 ms
Wall time: 788 ms


['Pengundi di Sungai Kandis menyaksikan penurunan mendadak peratusan keluar mengundi. Ini adalah yang kedua paling rendah selepas Pilihan Raya Umum ke-12 pada tahun 2009. Sebilangan besar pengundi atas pagar juga mungkin tidak memilih. Kegagalan untuk membentuk kerjasama padu dan sentimen yang dibawa oleh Barisan Nasional (BN) juga boleh menyebabkan kegagalan.',
 'Kurang dari faktor PRU mempengaruhi sentimen keluar mengundi, kata Dr Sivamurugan Pandian. Kegagalan BN-PAS membentuk kerjasama padu dan sentimen yang dibawa BN. Sentimen undi protes boleh diteliti dengan lebih mendalam, katanya.']

In [81]:
# input_ids = tokenizer.encode(f'ringkasan: {s}', return_tensors = 'pt').cuda()
# outputs = model.generate(input_ids,
#     max_length=256,
#     num_beams=5, 
#     no_repeat_ngram_size=2, 
#     num_return_sequences=5, 
#     early_stopping=True)

# small_outputs = small_model.generate(input_ids,
#     max_length=256,
#     num_beams=5, 
#     no_repeat_ngram_size=2, 
#     num_return_sequences=5, 
#     early_stopping=True)

# t = tokenizer.batch_decode(outputs, skip_special_tokens=True)
# t.extend(tokenizer.batch_decode(small_outputs, skip_special_tokens=True))
# t = [t_ for t_ in t if len(t_.split()) >= minimum_len]
# t = [postprocess(s, t_) for t_ in t]
# t = list(set(t))
# t